In [1]:
import numpy as np
import seaborn as sns
sns.set(style='whitegrid')
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

data = pd.read_excel('C:/Users/natha/Desktop/GitRepo/NathanLamplough/Data/credit_card.xls', delimiter = '\t', quoting = 0)
data = data[1:]
data = data.sample(frac=1).reset_index(drop=True)

In [2]:
X = data.drop(labels=['Y'], axis = 1).values
Y = data.Y.values
print(X[:5])
Y[:5]

[[20000 1 3 2 29 -1 -1 -1 -1 -1 -1 4453 7997 6573 7422 4381 11718 8019
  6779 7547 4589 12000 7556]
 [250000 1 1 1 46 -1 -1 -1 -1 -1 -1 1197 -19 3121 2372 502 1979 0 3140
  2387 504 1989 19150]
 [90000 2 2 2 30 0 0 0 0 2 2 75382 73980 75036 77094 75708 76610 3350
  3300 5500 1600 3950 0]
 [110000 2 1 2 26 0 0 0 0 -2 -2 104804 107979 93600 0 0 0 4988 1872 0 0 0
  0]
 [160000 2 1 1 47 1 2 0 0 2 0 89006 86694 88898 94289 91875 94405 0 4500
  7800 0 4000 3700]]


array([0, 1, 1, 0, 1], dtype=object)

In [3]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size = 0.33)


In [4]:
def min_max_normalized(data):
    col_max = np.max(data, axis=0)
    col_min = np.min(data, axis=0)
    return np.divide(data - col_min, col_max - col_min)

In [5]:
x_train = min_max_normalized(x_train)
x_test = min_max_normalized(x_test)

In [8]:
inp = tf.Variable(tf.random_normal(shape=[23,1]))
b = tf.Variable(tf.random_normal(shape=[1,1]))

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [10]:
data = tf.placeholder(dtype = tf.float32, shape=[None, 23])
target = tf.placeholder(dtype = tf.float32, shape=[None,1])
mod = tf.matmul(data, inp) + b

In [11]:
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=mod, labels = target))

In [12]:
learning_rate = 0.003
batch_size = 30
iter_num = 100

In [13]:
opt = tf.train.GradientDescentOptimizer(learning_rate)

In [14]:
goal = opt.minimize(loss)

In [16]:
prediction = tf.round(tf.sigmoid(mod))

correct = tf.cast(tf.equal(prediction, target), dtype = tf.float32)
accuracy = tf.reduce_mean(correct)

In [17]:
loss_trace = []
train_acc = []
test_acc = []

In [20]:
for epoch in range(iter_num):
    batch_index = np.random.choice(len(x_train), size = batch_size)
    batch_train_X = x_train[batch_index]
    batch_train_Y = np.matrix(y_train[batch_index]).T
    sess.run(goal, feed_dict={data: batch_train_X, target: batch_train_Y})
    temp_loss = sess.run(loss, feed_dict={data: batch_train_X, target: batch_train_Y})
    temp_train_acc = sess.run(accuracy, feed_dict={data: x_train, target: np.matrix(y_train).T})
    temp_test_acc = sess.run(accuracy, feed_dict={data: x_test, target: np.matrix(y_test).T})
    
    loss_trace.append(temp_loss)
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    
    if(epoch + 1) % 100 == 0:
        print('epoch: {:4d} loss: {:5f} train_acc: {:5f} test_acc: {:5f}'.format(epoch + 1, temp_loss,
                                                                          temp_train_acc, temp_test_acc))

epoch:  100 loss: 0.964892 train_acc: 0.279851 test_acc: 0.291010
